Работа с .npy форматом, загрузка 

In [30]:
!python3 test.py

Namespace(model_name='main_model', language='eng', bottleneck_bits=512, char_num=52, ref_nshot=4, batch_size=64, batch_size_val=8, img_size=64, max_seq_len=51, dim_seq=12, dim_seq_short=9, hidden_size=512, dim_seq_latent=512, ngf=16, n_aux_pts=6, random_index='00', name_ckpt='600_192921.ckpt', init_epoch=0, n_epochs=800, n_samples=20, lr=0.0002, ref_char_ids='0,1,26,27', mode='train', multi_gpu=False, name_exp='dvf', data_root='./data/vecfont_dataset/', freq_ckpt=50, freq_sample=500, freq_log=50, freq_val=500, beta1=0.9, beta2=0.999, eps=1e-08, weight_decay=0.0, tboard=True, kl_beta=0.01, loss_w_pt_c=0.01, loss_w_l1=10.0, loss_w_cmd=1.0, loss_w_args=1.0, loss_w_aux=0.01, loss_w_smt=10.0)


In [7]:
from dataloader import SVGDataset
import torchvision.transforms as T
import torch.utils.data as data

def set_range(X):
    return 1. - X

SetRange = T.Lambda(set_range)  # convert [0, 1] -> [0, 1]
transform = T.Compose([SetRange])
dataset = SVGDataset("root", 64, 'eng', 52, 51, 12, transform, 'train')

class_data = dataset[0]['seq_len'] # чтобы обратиться по ключу надо взять 0 индекс

dataloader = data.DataLoader(dataset, 1, shuffle= 'train', num_workers=0) # торчовская функция 

for idx, data in enumerate(dataloader):
    for key, value in data.items():
        print(f"Key: {key}, Shape: {value.shape}")
                

Finished loading train paths, number: 1
Key: class, Shape: torch.Size([1, 52, 1])
Key: seq_len, Shape: torch.Size([1, 52, 1])
Key: sequence, Shape: torch.Size([1, 52, 51, 12])
Key: pts_aux, Shape: torch.Size([1, 52, 51, 6])
Key: rendered, Shape: torch.Size([1, 52, 64, 64])


In [5]:
import torch
tensor = torch.zeros(2, 3, 4)
com = tensor[:,:,:1]
print(com.shape)

torch.Size([2, 3, 1])


In [ ]:
import torch

tensor = torch.zeros()